In [1]:
import datetime
import re
import numpy as np

from pyspark.sql.types import *
from pyspark.sql import functions as F

from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

In [2]:
%run ./0-Config

In [3]:
%run ./0-Functions

In [4]:
%run ./1-Heap-Common

#### Sample notebook to show some prep tasks
Show some examples of:
1. Removing unneeded data
2. Addressing data quality - nulls/NAs, duplicate rows
3. Normalizing data

Possible added topics here:
- Encoding

#### Useful general references

TDSP Data Prep: https://docs.microsoft.com/en-us/azure/machine-learning/team-data-science-process/prepare-data

Spark Python API Docs: https://spark.apache.org/docs/latest/api/python/index.html

Great Spark summary blog post: https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/ (a little outdated but still very good)

#### Start by getting some data to work with

Sample data - we'll use some data from staging1. To make it interesting let's get all data so far.

We'll use Heap pageviews.

In [8]:
# Variables to reduce the amount of hard-coding below

data_root = "data/"
folder_match = "views"

schema = schema_views

In [9]:
source_folders_raw = GetFoldersRecursive(adls2uri_staging1 + data_root)

source_folders = list(filter(lambda x: bool(re.search(folder_match, x)), source_folders_raw))

# print(source_folders)

In [10]:
df_all = spark.createDataFrame([], schema)

df_all.cache()

In [11]:
for source_folder in source_folders:
  df = spark\
    .read\
    .format("parquet")\
    .schema(schema)\
    .load(source_folder)
  
  print(source_folder + " | " + str(df.count()))

  df_all = df_all.union(df)

#### EDA

Acronym alert: Exploratory Data Analysis

In [13]:
count_all = df_all.count()

In [14]:
# Count - and distinct (fast way to find if there are 100% duplicate rows is if these differ)
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.count
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct

print("Dataframe rows: " + str(count_all))
print("Dataframe -distinct- rows: " + str(df_all.distinct().count()))

In [15]:
display(df_all)

In [16]:
# Basic descriptive stats by column
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe

display(df_all.describe())

In [17]:
# Additional descriptive stats - i.e. more than describe()
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.summary

display(df_all.summary())

In [18]:
# Plan explanation
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.explain

df_all.explain(extended = True)

In [19]:
# This shows the schema we created and provided explicitly at dataframe ingest from storage

df_all.printSchema()

In [20]:
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dtypes

df_all.dtypes

In [21]:
# Pair-wise frequency of columns
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.crosstab

display(df_all.crosstab("country", "language").sort(["country_language"]))

#### Remove unneeded data

Columns:
1. Drop columns from a dataframe: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.drop (subtractive)
2. Select columns from a dataframe: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.select (zero-additive)

Rows:
1. Select rows with a predicate

In [23]:
print("All data column count: " + str(len(df_all.columns)))

In [24]:
# Drop columns - as with all dataframe operations, the output is a new dataframe. The dataframe being operated on is unchanged.
# This means we can keep working with either df_all below, or with the df_drop we are creating here FROM df_all.

df_drop = df_all.drop("useless_column_1", "useless_column_2", "useless_column_3")

In [25]:
print("Drop data column count: " + str(len(df_drop.columns)))

In [26]:
# Select columns - again, this creates a new dataframe

df_select = df_all.select("useful_column_1", "useful_column_2", "useful_column_3")

In [27]:
print("Select data column count: " + str(len(df_select.columns)))

In [28]:
# Create (or replace, if we had already created it previously) a view which we can then use in Spark SQL statements

df_all.createOrReplaceTempView("v_all")

In [29]:
# Here, we use a Spark SQL query to project a new dataframe from the view we just created
# We could use %sql (i.e. SQL magic) to write SQL directly into a cell, but running it like this allows us to create a query dynamically
# using variables set elsewhere, for example. We can also use this approach when we want to use custom predicates instead of generic functionality
# like dropna (see below).

useful_1 = "United States"

df_sql = spark.sql("""
SELECT
useful_column_1, useful_column_2, useful_column_3, useless_column_1, useless_column_2, useless_column_3
FROM v_all
WHERE useful_column_1 = '""" + useful_1 + """'
""")

In [30]:
df_sql.count()

In [31]:
display(df_sql)

#### Data Quality

- Duplicate rows - remove them if they are inappropriate
- Nulls/NAs - drop or replace them

##### Duplicate rows

In [34]:
# Let's say we have decided that any 100% duplicate rows - i.e. every field in a row has the identical value as in one or more other rows - are not legitimately in our dataset. (Sometimes 100% duplicate rows MAY be valid - this decision requires business understanding, or at least enough data understanding to know, for example, if duplicates are exclusively an unavoidable and undesirable byproduct of a join, for example, and therefore OK to remove even if we don't have full domain understanding of all fields' meaning.)

# We already checked for 100% duplicates above by comparing df_all.count() to df_all.distinct().count(). If there are duplicates, the second number will be less than the first number.

# The dataframe dropDuplicates() method optionally allows us to consider only a subset of all columns to determine duplicate rows. Here, we will just go default, not specify a subset, and use all fields to determine duplication.
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dropDuplicates

df_all_deduped = df_all.dropDuplicates()

count_all_deduped = df_all_deduped.count()
count_removed = count_all - count_all_deduped

In [35]:
print("Count all: " + str(count_all))
print("Count deduped: " + str(count_all_deduped))
print("Count removed: " + str(count_removed))

##### Nulls/NAs

In [37]:
# Now lets' drop rows with nulls. We can choose to drop rows with any nulls, or rows where ALL fields are null, or rows with less than some number of non-null fields. By default, all columns will be considered, and we can optionally pass a subset of columns to consider instead of all.

# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dropna

# We will do these: 
# - drop rows where ANY columns are null
# - drop rows where ALL columns are null
# - drop rows where country is null

# Then we'll compare counts.

df_all_drop_nulls_any = df_all.dropna(how = "any")
df_all_drop_nulls_all = df_all.dropna(how = "all")
df_all_drop_nulls_country = df_all.dropna(subset = "country")

In [38]:
count_all_drop_nulls_any = df_all_drop_nulls_any.count()
count_all_drop_nulls_all = df_all_drop_nulls_all.count()
count_all_drop_nulls_country = df_all_drop_nulls_country.count()

In [39]:
print("Count all: " + str(count_all))
print("Count with rows containing ANY nulls removed: " + str(count_all_drop_nulls_any))
print("Count with rows containing ALL nulls removed: " + str(count_all_drop_nulls_all))
print("Count with rows where country is null removed: " + str(count_all_drop_nulls_country))
print("Number of rows where country is null: " + str(count_all - count_all_drop_nulls_country))

In [40]:
# Let's consider the case where we do NOT want to drop rows if there are missing values. Instead, we want to substitute something for
# the missing values. We can take a simple approach using Spark dataframe API's fillna(), or we can use imputation.

# Simplistic imputation: https://stackoverflow.com/questions/37424942/pyspark-dataframe-imputations-replace-unknown-missing-values-with-column-me
# Using sk-learn: https://machinelearningmastery.com/handle-missing-data-python/ - numeric data

# Here we will use fillna
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.fillna

# Value to fill in for null country
fillna_useful_column_1 = "Erewhon"

df_all_fillna_useful_column_1 = df_all.fillna(fillna_useful_column_1, subset = "useful_column_1")

# Count how many rows now have the fill in value. This should be the same as the number of rows where country is null in preceding cell.
df_all_fillna_useful_column_1.filter("useful_column_1 == '" + fillna_useful_column_1 + "'").count()

#### Normalization / scaling

Spark docs on feature extractors, transformers, and selectors: https://spark.apache.org/docs/latest/ml-features.html

In [42]:
# Prepare a schema to be used in operations outputting normalized data
# Here we will be using event_id as the key to match rows across dataframes,
# and we'll be normalizing the pageversion column - I didn't say this was a real-world example :)
schema_features_normalized = StructType([
  StructField("id", LongType(), True),
  StructField("useful_column_1", FloatType(), True)
])

In [43]:
# Convert the column we'll normalize from string (ingested as such) to integer so we can normalize it

df_all_norm = df_all\
  .withColumn("useful_column_2a", df_all.useful_column_2.cast(IntegerType()))\
  .drop("useful_column_2")\
  .withColumnRenamed("useful_column_2a", "useful_column_2")\
  .fillna(0, "useful_column_2)

In [44]:
# Utility function to use in normalization

def extract(row):
    return (row.event_id, ) + tuple(row.features_scaled.toArray().tolist())

In [45]:
# Use a MinMax assembler to transform the pageversion column
# Good blog post: https://medium.com/@connectwithghosh/basic-data-preparation-in-pyspark-capping-normalizing-and-scaling-252ee7acba7d

assembler = VectorAssembler().setInputCols(["useful_column_1", "useful_column_2"]).setOutputCol("features")
transformed = assembler.transform(df_all_norm)
scaler = MinMaxScaler(inputCol = "features", outputCol = "features_scaled")
scaler_model = scaler.fit(transformed.select("features"))
scaled_data = scaler_model.transform(transformed)

In [46]:
# Output the key column and scaled feature column(s) - this is where we use the above schema so that our columns are named

df_features_normalized = scaled_data\
  .select("useful_column_3", "features_scaled")\
  .rdd\
  .map(extract)\
  .toDF(schema_features_normalized)

In [47]:
display(df_features_normalized)

In [48]:
# Join the dataframe with the normalized column back to the original dataframe
# This syntax avoids duplicated join column(s)
# The resultant dataframe will have both the original column as well as the normalized column.
# Drop the original column at this point, since it will not be needed for ML modeling

df_all_normalized = df_all\
  .join(df_features_normalized, "id")\
  .drop("useful_column_3")


In [49]:
display(df_all_normalized)